In [1]:
import surprise
## 고유값(특이값)분해 방법
from surprise import SVD
## SGD(Stochastic 통계적인 Gradient 경사 하강법)
from surprise import Dataset, Reader
from surprise import accuracy
from surprise.model_selection import train_test_split
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
import pandas as pd
import numpy as np

In [2]:
def get_dataframe():
    rating = pd.read_csv('ratings_trip.csv', index_col=0)
    area = pd.read_csv('place_trip.csv', index_col=0)
    user = pd.read_csv('user_trip.csv', index_col=0)
    return user, area, rating

In [3]:
def cal_RMSE(rating):
    algo = SVD()
    reader = Reader(rating_scale=(1,5.0)) # 1단위, 최대값 5
    data = Dataset.load_from_df(rating[['userID', 'area', 'rating']], reader)
    trainset, testset = train_test_split(data, test_size=0.25, random_state=0)
    algo.fit(trainset)
    predictions = algo.test(testset)
    accuracy.rmse(predictions)
    return (algo, trainset, testset)

In [4]:
def get_recommendations(userID, algo, user, rating, num_of_ppl=50):
    # model 호출    
    # 간단 전처리
    user_df = pd.get_dummies(user)
    col_data = user_df.iloc[:,1:].columns
    user_matrix = user_df.iloc[:,1:].to_numpy()
    cosine_sim = cosine_similarity(user_matrix,user_matrix)
    
    # 선택한 유저의 id로부터 해당 유저의 인덱스를 받아온다.
    userID_to_index = dict(zip(user_df['userID'], user.index))
    idx = userID_to_index[userID]

    # 해당 유저와 모든 유저와의 유사도를 가져온다.
    sim_scores = list(enumerate(cosine_sim[idx]))

    # 유사도에 따라 유저들을 정렬한다.
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # 가장 유사한 50개의 유저를 받아온다.
    sim_scores = sim_scores[1:num_of_ppl+1]

    # 가장 유사한 10개의 유저의 인덱스를 얻는다.
    user_indices = [idx[0] for idx in sim_scores]

    # 가장 유사한 10개의 유저의 id을 r_list에 저장.
    rlist = list(user_df['userID'].iloc[user_indices])
    print("가장 유사한",num_of_ppl,"명의 userID:\n", rlist)
    ##비슷한 유저 num_of_ppl명
    similar_users = user_df[user_df['userID'].isin(rlist)]
    
    similarity = cosine_similarity(similar_users, similar_users)
    print("\n\n")
    print("비슷한 유저",num_of_ppl,"명을 비교한 cosine similarity:\n", similarity)
    
    ## 비슷한 사람들 20명이 4이상 점수를 준 여행지역
    df_sim = rating[(rating['userID'].isin(rlist)) & (rating['rating']>=4)]
    print("\n\n비슷한 사람들",num_of_ppl,"명이 4이상 점수를 준 여행지역:\n", df_sim)

    ## 유저가 여행지역에 줄 점수 예측
    result= [(algo.predict(userID, str(one), verbose=False)) for one in df_sim['area']]
    print("\n\nuserID:",userID,"유저가 여행지역에 줄 점수 예측:")
    for i in result:
        print(i)
    
    result_est = [pred.est for pred in result]
    result_est = list(set(result_est))
    # print(result_est)
    
    ## 여행지역중 최대 평점 예측되는 지역 top 5
    top5 = sorted(range(len(result_est)), key=lambda i: result_est[i])[-5:]
    top5.reverse()
    
    ## top5여행지에 대한 유저 평가 예측
    recommendation = []
    print("\n\ntop5 여행지에 대한 유저 평가 예측:")
    for i in top5:
        print(result[i])
        recommendation.append(result[i][1])
        
    print("\n\n추천여행지 5곳:")
    print(recommendation)
    return recommendation


In [5]:
## get_dateframe()함수 호출하여 dataframe 저장
user, area, rating = get_dataframe()

In [6]:
## model, train_test_split된 df 받고 RMSE값 계산
algo, trainset, testset = cal_RMSE(rating)

RMSE: 0.4738


In [7]:
## 추천받기
# get_recommendations(1000001, algo, 50) ## userID, 

In [8]:
# pearson_sim = user_df.T.corr()
# np.array(pearson_sim)

In [9]:
 ## 유사한 유저 50명 정보 출력
# user[user_df['userID'].isin(rlist)]

In [10]:
# import seaborn as sns
# from matplotlib import pyplot as plt

# sns.heatmap(cosine_sim, xticklabels=similar_users['userID'], yticklabels=similar_users['userID'], cmap='viridis')
# plt.show()

## ADD NEW USER

In [11]:
new = {
    'userID':[1000001],
    'sex':['남자'],
    'age':[20],
    'living_area':['서울특별시'],
    'living_area_size':['중소도시'],
    'occupation':['대학생'],
    'family_size':[4],
    'marry':['미혼'],
    'income_per_year':[10000]
}

In [12]:
new_list = [*new.values()]

In [13]:
new_list

[[1000001], ['남자'], [20], ['서울특별시'], ['중소도시'], ['대학생'], [4], ['미혼'], [10000]]

In [14]:
new_list = [1000001,'남자',20,'서울특별시','중소도시','대학생',4, '미혼',10000]

In [15]:
new_df = pd.DataFrame(new)
new_df

,userID,sex,age,living_area,living_area_size,occupation,family_size,marry,income_per_year
0,1000001,남자,20,서울특별시,중소도시,대학생,4,미혼,10000


In [16]:
df = pd.concat([user, new_df], ignore_index = True, axis = 0)
df

,userID,sex,age,living_area,living_area_size,occupation,family_size,marry,income_per_year
0,1000101,남성,40,서울특별시,대도시,전문가 및 관련 종사자,4,배우자 있음,4000
1,1000102,여성,40,서울특별시,대도시,가정주부,4,배우자 있음,4000
2,1000201,남성,41,서울특별시,대도시,서비스 종사자,3,미혼,4000
3,1000202,여성,73,서울특별시,대도시,가정주부,3,사별,4000
4,1000203,여성,46,서울특별시,대도시,단순노무자,3,배우자 있음,4000
...,...,...,...,...,...,...,...,...,...
6166,1539501,남성,51,경상남도,중소도시,판매 종사자,3,배우자 있음,7000
6167,1539502,여성,39,경상남도,중소도시,관리자,3,배우자 있음,7000
6168,1539901,남성,39,부산광역시,중소도시,사무 종사자,4,배우자 있음,9000
6169,1539902,여성,38,부산광역시,중소도시,가정주부,4,배우자 있음,9000


In [17]:
user.at[len(user.index)] = new_list

In [18]:
user

,userID,sex,age,living_area,living_area_size,occupation,family_size,marry,income_per_year
0,1000101.0,남성,40.0,서울특별시,대도시,전문가 및 관련 종사자,4.0,배우자 있음,4000.0
1,1000102.0,여성,40.0,서울특별시,대도시,가정주부,4.0,배우자 있음,4000.0
2,1000201.0,남성,41.0,서울특별시,대도시,서비스 종사자,3.0,미혼,4000.0
3,1000202.0,여성,73.0,서울특별시,대도시,가정주부,3.0,사별,4000.0
4,1000203.0,여성,46.0,서울특별시,대도시,단순노무자,3.0,배우자 있음,4000.0
...,...,...,...,...,...,...,...,...,...
6166,1539501.0,남성,51.0,경상남도,중소도시,판매 종사자,3.0,배우자 있음,7000.0
6167,1539502.0,여성,39.0,경상남도,중소도시,관리자,3.0,배우자 있음,7000.0
6168,1539901.0,남성,39.0,부산광역시,중소도시,사무 종사자,4.0,배우자 있음,9000.0
6169,1539902.0,여성,38.0,부산광역시,중소도시,가정주부,4.0,배우자 있음,9000.0


## get recommendation

In [19]:
get_recommendations(1000001, algo, user, rating, 50)

가장 유사한 50 명의 userID:
 [1516005.0, 1516004.0, 1006204.0, 1220603.0, 1013604.0, 1037603.0, 1146903.0, 1519704.0, 1521504.0, 1536804.0, 1050203.0, 1054904.0, 1200705.0, 1047804.0, 1207604.0, 1235404.0, 1521503.0, 1534803.0, 1278504.0, 1503605.0, 1129204.0, 1174304.0, 1276905.0, 1052803.0, 1176704.0, 1523305.0, 1006203.0, 1097304.0, 1006205.0, 1235505.0, 1537004.0, 1132804.0, 1224704.0, 1224705.0, 1071304.0, 1220604.0, 1169405.0, 1036804.0, 1129003.0, 1263204.0, 1189003.0, 1224004.0, 1276904.0, 1143703.0, 1159604.0, 1120803.0, 1092103.0, 1120804.0, 1074704.0, 1276704.0]



비슷한 유저 50 명을 비교한 cosine similarity:
 [[1.         1.         1.         ... 0.99998537 0.99998532 0.99998158]
 [1.         1.         1.         ... 0.99998537 0.99998532 0.99998158]
 [1.         1.         1.         ... 0.99998537 0.99998532 0.99998158]
 ...
 [0.99998537 0.99998537 0.99998537 ... 1.         1.         0.99999978]
 [0.99998532 0.99998532 0.99998532 ... 1.         1.         0.99999979]
 [0.99998158 0.99

['강원도 속초시', '대구광역시 서구', '강원도 강릉시', '경상북도 경주시', '경상남도 거제시']

In [20]:
rating

,userID,rating,area
0,1000101,4.0,경기도 수원시
1,1000102,4.0,경기도 양주시
2,1000203,4.0,강원도 강릉시
5,1000303,4.0,인천광역시 연수구
7,1000401,4.0,강원도 속초시
...,...,...,...
11296,1539203,4.0,부산광역시 부산진구
11297,1539203,4.0,부산광역시 부산진구
11298,1539202,5.0,전라북도 전주시
11299,1539203,3.0,경상남도 의령군


In [21]:
user_df = pd.get_dummies(user)
col_data = user_df.iloc[:,1:].columns
user_matrix = user_df.iloc[:,1:].to_numpy()
cosine_sim = cosine_similarity(user_matrix,user_matrix)
cosine_sim

array([[1.        , 0.99999988, 0.99999981, ..., 0.99998366, 0.999983  ,
        0.99996767],
       [0.99999988, 1.        , 0.99999975, ..., 0.99998363, 0.99998305,
        0.99996767],
       [0.99999981, 0.99999975, 1.        , ..., 0.99998229, 0.9999816 ,
        0.99996578],
       ...,
       [0.99998366, 0.99998363, 0.99998229, ..., 1.        , 0.99999997,
        0.99999723],
       [0.999983  , 0.99998305, 0.9999816 , ..., 0.99999997, 1.        ,
        0.99999749],
       [0.99996767, 0.99996767, 0.99996578, ..., 0.99999723, 0.99999749,
        1.        ]])

In [22]:
user_matrix = user_df.iloc[:,1:].to_numpy()
user_matrix

array([[4.0e+01, 4.0e+00, 4.0e+03, ..., 1.0e+00, 0.0e+00, 0.0e+00],
       [4.0e+01, 4.0e+00, 4.0e+03, ..., 1.0e+00, 0.0e+00, 0.0e+00],
       [4.1e+01, 3.0e+00, 4.0e+03, ..., 0.0e+00, 0.0e+00, 0.0e+00],
       ...,
       [3.9e+01, 4.0e+00, 9.0e+03, ..., 1.0e+00, 0.0e+00, 0.0e+00],
       [3.8e+01, 4.0e+00, 9.0e+03, ..., 1.0e+00, 0.0e+00, 0.0e+00],
       [2.0e+01, 4.0e+00, 1.0e+04, ..., 0.0e+00, 0.0e+00, 0.0e+00]])

In [23]:
user_df.loc[

SyntaxError: unexpected EOF while parsing (3924805938.py, line 1)